In [40]:
import kaggle as kg
import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils import to_categorical

from keras.layers import Input, Dense, BatchNormalization
from keras.models import Model
from keras.optimizers import SGD,RMSprop,Adam

In [2]:
os.environ["KAGGLE_USERNAME"] = "thasinkhan"
os.environ["KAGGLE_KEY"] = "d0e58b5e1972f61220788d36914159c2"

In [3]:
kg.api.authenticate()


In [4]:
kg.api.dataset_download_files(dataset="medahmedkrichen/devanagari-handwritten-character-datase",
                              path="dataset",unzip=True)

Dataset URL: https://www.kaggle.com/datasets/medahmedkrichen/devanagari-handwritten-character-datase


In [5]:
def train_test_df(path):

    img_path = list()
    img_label = list()

    for single_class_dir_path in pathlib.Path(path).glob("*"):

        for single_class_img_path in pathlib.Path(single_class_dir_path).glob("*.png"):

            img_path.append(str(single_class_img_path))
            #print(str(single_class_img_path).split("/")[-2].split("_")[-1])
            img_label.append(str(single_class_img_path).split("/")[-2].split("_")[-1])

    return pd.DataFrame(data={"img_path":img_path,"label":img_label})    

In [6]:
train_path = "dataset/DevanagariHandwrittenCharacterDataset/Train"
test_path = "dataset/DevanagariHandwrittenCharacterDataset/Test"

In [7]:
training_data = train_test_df(train_path)
testing_data = train_test_df(test_path)

In [8]:
training_data.shape,testing_data.shape

((78200, 2), (13800, 2))

In [9]:
training_data.head()

,img_path,label
0,dataset/DevanagariHandwrittenCharacterDataset/...,ra
1,dataset/DevanagariHandwrittenCharacterDataset/...,ra
2,dataset/DevanagariHandwrittenCharacterDataset/...,ra
3,dataset/DevanagariHandwrittenCharacterDataset/...,ra
4,dataset/DevanagariHandwrittenCharacterDataset/...,ra


In [10]:
testing_data.head()

,img_path,label
0,dataset/DevanagariHandwrittenCharacterDataset/...,ra
1,dataset/DevanagariHandwrittenCharacterDataset/...,ra
2,dataset/DevanagariHandwrittenCharacterDataset/...,ra
3,dataset/DevanagariHandwrittenCharacterDataset/...,ra
4,dataset/DevanagariHandwrittenCharacterDataset/...,ra


In [11]:
training_data["label"].unique()

array(['ra', 'chha', 'patalosaw', 'dha', 'la', 'ga', 'yaw', 'na', '3',
       'petchiryakha', 'pha', 'gya', 'da', 'kha', 'tha', '1', 'ja', '4',
       'ma', 'motosaw', 'chhya', 'ha', '6', 'taamatar', '2', 'yna',
       'adna', 'tra', 'kna', 'tabala', 'ba', 'waw', '7', 'pa', '0', 'jha',
       'bha', 'daa', '9', '8', 'dhaa', 'gha', 'cha', 'thaa', 'ka', '5'],
      dtype=object)

In [12]:
range(len(training_data['label'].unique()))

range(0, 46)

In [13]:
character2int = dict(zip(training_data["label"].unique(),range(len(training_data["label"].unique()))))

In [14]:
character2int.keys()

dict_keys(['ra', 'chha', 'patalosaw', 'dha', 'la', 'ga', 'yaw', 'na', '3', 'petchiryakha', 'pha', 'gya', 'da', 'kha', 'tha', '1', 'ja', '4', 'ma', 'motosaw', 'chhya', 'ha', '6', 'taamatar', '2', 'yna', 'adna', 'tra', 'kna', 'tabala', 'ba', 'waw', '7', 'pa', '0', 'jha', 'bha', 'daa', '9', '8', 'dhaa', 'gha', 'cha', 'thaa', 'ka', '5'])

In [15]:
training_data["label"].replace(to_replace=character2int.keys(),value=character2int.values(),
                               inplace=True)

testing_data.replace(to_replace=character2int.keys(),value=character2int.values(),
                     inplace=True)

In [16]:
Y_true_train = to_categorical(y=training_data["label"],num_classes=46)
Y_true_test = to_categorical(y=testing_data["label"],num_classes=46)

In [17]:
Y_true_test,Y_true_train

(array([[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]], dtype=float32),
 array([[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]], dtype=float32))

In [18]:
Y_true_train.shape

(78200, 46)

In [19]:
def multiclass_dnn():

    input_to_dnn = Input(shape=(1024,))
    first_dense_out = Dense(units=1024,activation="relu")(input_to_dnn)
    second_dense_out = Dense(units=1024,activation="relu")(first_dense_out)
    second_dense_out = BatchNormalization()(second_dense_out)
    output = Dense(units=46,activation="softmax")(second_dense_out)

    return Model(inputs=[input_to_dnn],outputs=[output])

In [34]:
def custom_data_generator(data_df,Y_true,mb_size):
    
    for time_step in range(data_df.shape[0]//mb_size):
        X_mb = list()
        for img_path in data_df.iloc[time_step*mb_size:(time_step+1)*mb_size,0]:
            img_np_array = plt.imread(img_path)
            reshaped_np_array = img_np_array.reshape(1024,)
            X_mb.append(reshaped_np_array)
        X_mb = np.array(X_mb)
        Y_true_mb = Y_true[time_step*mb_size:(time_step+1)*mb_size]
        
        yield X_mb, Y_true_mb

In [51]:
epochs = 10
training_data_mb_size = 782
testing_data_mb_size = 138

In [52]:
model = multiclass_dnn()
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1024)]            0         
                                                                 
 dense_6 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_7 (Dense)             (None, 1024)              1049600   
                                                                 
 batch_normalization_2 (Bat  (None, 1024)              4096      
 chNormalization)                                                
                                                                 
 dense_8 (Dense)             (None, 46)                47150     
                                                                 
Total params: 2150446 (8.20 MB)
Trainable params: 2148398 (8.20 MB)
Non-trainable params: 2048 (8.00 KB)
____________________

In [53]:
def loss_fn(Y_true_mb,Y_pred_mb):
    return tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true=Y_true_mb,y_pred=Y_pred_mb))
optimizer = SGD()

In [54]:
@tf.function
def training_step(X_train_mb,Y_true_train_mb):

    with tf.GradientTape() as tape:
            
        Y_pred_train_mb = model(X_train_mb, training=True)
        training_loss = loss_fn(Y_true_train_mb, Y_pred_train_mb)

    grads = tape.gradient(training_loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(Y_true_train_mb,Y_pred_train_mb)

    return training_loss

In [55]:
@tf.function
def testing_forward_pass(X_test_mb,Y_true_test_mb):

    Y_pred_test_mb = model(X_test_mb,training=False)
    testing_loss = loss_fn(Y_true_test_mb,Y_pred_test_mb)
    test_acc_metric.update_state(Y_true_test_mb,Y_pred_test_mb)

    return testing_loss

In [56]:
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

for epoch in range(epochs):

    training_data_generator = custom_data_generator(training_data,Y_true_train,782)

    for time_step, (X_train_mb, Y_true_train_mb) in enumerate(training_data_generator):
        training_loss = training_step(X_train_mb,Y_true_train_mb)

        if (time_step+1) % 50 == 0:
            print("Epoch %d, Time Step %d, Training loss for one mini batch: %.4f"
            % (epoch+1, time_step+1, float(training_loss)))
            
    training_acc = train_acc_metric.result()    
    print("Epoch %d, Training Accuracy: %.2f" % (epoch+1,float(training_acc)))
    train_acc_metric.reset_states()

    testing_data_generator = custom_data_generator(testing_data,Y_true_test,testing_data_mb_size)

    for X_test_mb, Y_true_test_mb in testing_data_generator:
        testing_loss = testing_forward_pass(X_test_mb,Y_true_test_mb)

    print("\nEpoch %d, Testing Loss for last mini batch: %.4f" % (epoch+1,float(testing_loss)))
    testing_acc = test_acc_metric.result()
    print("Epoch %d, Testing Accuracy: %.2f" % (epoch+1,float(testing_acc)))
    test_acc_metric.reset_states()

    print("\n\n")


Epoch 1, Time Step 50, Training loss for one mini batch: 4.3327
Epoch 1, Time Step 100, Training loss for one mini batch: 4.2772
Epoch 1, Training Accuracy: 0.03

Epoch 1, Testing Loss for last mini batch: 4.0263
Epoch 1, Testing Accuracy: 0.12



Epoch 2, Time Step 50, Training loss for one mini batch: 4.1985
Epoch 2, Time Step 100, Training loss for one mini batch: 4.1998
Epoch 2, Training Accuracy: 0.07

Epoch 2, Testing Loss for last mini batch: 3.8732
Epoch 2, Testing Accuracy: 0.20



Epoch 3, Time Step 50, Training loss for one mini batch: 4.1527
Epoch 3, Time Step 100, Training loss for one mini batch: 4.1578
Epoch 3, Training Accuracy: 0.09

Epoch 3, Testing Loss for last mini batch: 3.7261
Epoch 3, Testing Accuracy: 0.25



Epoch 4, Time Step 50, Training loss for one mini batch: 4.1278
Epoch 4, Time Step 100, Training loss for one mini batch: 4.1317
Epoch 4, Training Accuracy: 0.11

Epoch 4, Testing Loss for last mini batch: 3.6139
Epoch 4, Testing Accuracy: 0.28



Epoch 5,